In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

class WebScraper:
    def __init__(self, base_url, asset_type):
        self.base_url = base_url
        self.url_list = []
        self.asset_type = asset_type
        
    def fetch_cards(self):
        response = requests.get(self.base_url + self.asset_type)
        soup = BeautifulSoup(response.content, 'html.parser')
        cards = soup.find_all('div', class_='card get-single-page')
        for card in cards:
            p_tag = card.find('p')
            if p_tag:
                url_string = self.base_url + p_tag.get_text()
                self.url_list.append(url_string)

    def fetch_texts(self):
        import os
        if not os.path.exists('data'):
            os.makedirs('data')
        self.fetch_cards()
        all_texts = {}  # Dictionary to hold all texts

        for url in self.url_list:
            response = requests.get(url)
            soup = BeautifulSoup(response.content, 'html.parser')
            primary_element = soup.find(id='primary')
            if primary_element:
                primary_text = primary_element.get_text(strip=True).replace('Export PDF', '')
                column_name = url.split('/')[-1]
                all_texts[column_name] = primary_text
        pd.DataFrame.from_dict(all_texts, orient='index', columns=['Text']).to_csv('data/' + self.asset_type + '.csv', index_label='Name')

# Usage
WebScraper('https://infotechnology.fhwa.dot.gov/','bridge').fetch_texts()